In [20]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def generate_correlated_data(n_samples=1000,n_features=7,random_seed=42):
    np.random.seed(random_seed)
    X_base=np.random.rand(n_samples,1)*10
    X=X_base.copy()
    for i in range(n_features-1):
        X_correlated=X_base*(1+(i+1)*0.5)+np.random.normal(0,0.5,(n_samples,1))
        X=np.hstack((X,X_correlated))
    true_weights=np.array([5,-2,1.5,3,-1,0.5,4])
    bias=10
    y=X@true_weights+bias+np.random.normal(0,5,n_samples)
    return X,y
X,y=generate_correlated_data()
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
X_b=np.c_[np.ones((X_scaled.shape[0],1)),X_scaled]

X_train,X_test,y_train,y_test=train_test_split(
    X_b,y,test_size=0.3,random_state=42
)
class RidgeRegressionGD:
    def __init__(self,learning_rate=0.01,n_iterations=1000,lambda_param=1e-5):
        self.learning_rate=learning_rate
        self.n_iterations=n_iterations
        self.lambda_param=lambda_param
        self.theta=None
    def hypothesis(self,X):
        return X@self.theta
    def cost_function(self,X,y):
        m=len(y)
        mse=(1/(2*m))*np.sum((self.hypothesis(X)-y)**2)
        regularization=self.lambda_param*np.sum(self.theta[1:]**2)
        return mse+regularization
    def fit(self,X,y):
        m,n=X.shape
        self.theta=np.zeros(n)
        for _ in range(self.n_iterations):
            predictions=self.hypothesis(X)
            errors=predictions-y
            grad_0=(1/m)*np.sum(errors*X[:,0])
            grad_features=(1/m)*(X[:,1:].T@errors)+(self.lambda_param * self.theta[1:])
            self.theta[0]-=self.learning_rate * grad_0
            self.theta[1:]-=self.learning_rate * grad_features
        return self
    def predict(self,X):
        return self.hypothesis(X)
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]
lambda_param = 1e-5  
results = []
best_r2 = -np.inf
best_cost = np.inf
best_params = {}

for lr in learning_rates:
    model = RidgeRegressionGD(learning_rate=lr, n_iterations=1000, lambda_param=lambda_param)
    
    
    try:
        model.fit(X_train, y_train)
        
        y_pred_test = model.predict(X_test)
        
        
        if not np.all(np.isfinite(y_pred_test)):
            raise ValueError("Model Diverged: Predictions contain NaN/inf.")
            
        
        train_cost = model.cost_function(X_train, y_train)
        r2 = r2_score(y_test, y_pred_test) 
        
        results.append({
            'lr': lr,
            'lambda': lambda_param,
            'Train Cost': train_cost,
            'R2 Score': r2
        })

        print(f"LR: {lr:<6} | Cost: {train_cost:.4f} | R2: {r2:.4f}")

        

    except (OverflowError, ValueError) as e:
        
        print(f"LR: {lr:<6} | Model Failed or Diverged ({type(e).__name__})")
        results.append({'lr': lr, 'lambda': lambda_param, 'Train Cost': np.nan, 'R2 Score': np.nan})
        
        

        
        


print("\n---------------------------------------------------")
if best_params:
    print(f"Best Parameters Found:")
    print(f"Learning Rate (LR): {best_params['lr']}")
    print(f"Minimum Ridge Cost (Train): {best_params['cost']:.4f}")
    print(f"Maximum R2 Score (Test): {best_params['r2']:.4f}")
else:
    print("No stable model found.")
    

LR: 0.0001 | Cost: 9249.4203 | R2: -1.8360
LR: 0.001  | Cost: 1404.4974 | R2: 0.5549
LR: 0.01   | Cost: 17.7321 | R2: 0.9947
LR: 0.1    | Cost: 15.5167 | R2: 0.9954
LR: 1      | Model Failed or Diverged (ValueError)
LR: 10     | Model Failed or Diverged (ValueError)

---------------------------------------------------
No stable model found.


C:\Users\hp\AppData\Local\Temp\ipykernel_12444\3676403224.py:45: RuntimeWarning: overflow encountered in matmul
  grad_features=(1/m)*(X[:,1:].T@errors)+(self.lambda_param * self.theta[1:])
C:\Users\hp\AppData\Local\Temp\ipykernel_12444\3676403224.py:32: RuntimeWarning: invalid value encountered in matmul
  return X@self.theta
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [8]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score


df = pd.read_csv(r"C:\Users\hp\Downloads\Hitters (1).csv")




df.dropna(subset=['Salary'], inplace=True) 

df.dropna(inplace=True) 


X = df.drop('Salary', axis=1)
y = df['Salary']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()


preprocessor = ColumnTransformer(
    transformers=[
        
        ('num', StandardScaler(), numerical_features), 
        
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) 
    ])



alpha_val = 0.5748 


linear_model = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', LinearRegression())])
linear_model.fit(X_train, y_train)


ridge_model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', Ridge(alpha=alpha_val, random_state=42))])
ridge_model.fit(X_train, y_train)


lasso_model = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', Lasso(alpha=alpha_val, random_state=42, max_iter=10000))])
lasso_model.fit(X_train, y_train)



models = {
    'Linear Regression': linear_model,
    'Ridge Regression (L2)': ridge_model,
    'LASSO Regression (L1)': lasso_model
}

evaluation_results = []

for name, model in models.items():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    evaluation_results.append({'Model': name, 'MSE': mse, 'RMSE': rmse, 'R2 Score': r2})

results_df = pd.DataFrame(evaluation_results)



print(results_df.sort_values(by='R2 Score', ascending=False).to_markdown(index=False, floatfmt=".4f"))

| Model                 |         MSE |     RMSE |   R2 Score |
|:----------------------|------------:|---------:|-----------:|
| Ridge Regression (L2) | 145189.5482 | 381.0375 |     0.4021 |
| LASSO Regression (L1) | 146328.8781 | 382.5296 |     0.3974 |
| Linear Regression     | 150406.5837 | 387.8229 |     0.3806 |


In [14]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer 


#print("Loading Boston Housing Dataset")
try:
    
    boston = fetch_openml(name='boston', version=1, as_frame=True, parser='auto')
    X = boston.data
    y = boston.target
    
    
    if X.isnull().any().any():
        print("Handling missing values (Imputing mean for numerical data).")
        
        
except Exception as e:
    print(f"Error loading Boston dataset: {e}. Cannot proceed with Q3.")
    


numerical_features = X.select_dtypes(include=np.number).columns.tolist()

categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist() 


preprocessor = ColumnTransformer(
    transformers=[
        
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        
        
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)


X_processed = preprocessor.fit_transform(X)



X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)


alphas = np.logspace(-4, 2, 100) 



print("\nRidge Cross Validation (RidgeCV)")

ridge_cv = RidgeCV(alphas=alphas, cv=5) # 5-fold CV
ridge_cv.fit(X_train, y_train)


best_alpha_ridge = ridge_cv.alpha_
r2_ridge = ridge_cv.score(X_test, y_test)

print(f"Optimal Alpha (Lambda) for RidgeCV: {best_alpha_ridge:.4f}")
print(f"RidgeCV Test R2 Score: {r2_ridge:.4f}")



print("\n Lasso Cross Validation (LassoCV)")

lasso_cv = LassoCV(n_alphas=100, cv=5, max_iter=10000, random_state=42)
lasso_cv.fit(X_train, y_train)


best_alpha_lasso = lasso_cv.alpha_
r2_lasso = lasso_cv.score(X_test, y_test)
non_zero_coeffs = np.sum(lasso_cv.coef_ != 0)

print(f"Optimal Alpha (Lambda) for LassoCV: {best_alpha_lasso:.4f}")
print(f"LassoCV Test R2 Score: {r2_lasso:.4f}")
print(f"Number of non-zero coefficients (Feature Selection): {non_zero_coeffs}")


Ridge Cross Validation (RidgeCV)
Optimal Alpha (Lambda) for RidgeCV: 10.7227
RidgeCV Test R2 Score: 0.7071

 Lasso Cross Validation (LassoCV)
Optimal Alpha (Lambda) for LassoCV: 0.0070
LassoCV Test R2 Score: 0.7137
Number of non-zero coefficients (Feature Selection): 20


c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1622: FutureWarning: 'n_alphas' was deprecated in 1.7 and will be removed in 1.9. 'alphas' now accepts an integer value which removes the need to pass 'n_alphas'. The default value of 'alphas' will change from None to 100 in 1.9. Pass an explicit value to 'alphas' and leave 'n_alphas' to its default value to silence this warning.
  warnings.warn(


In [18]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


iris = load_iris()
X = iris.data
y = iris.target
n_classes = len(np.unique(y)) 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


X_train_b = np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]
X_test_b = np.c_[np.ones((X_test_scaled.shape[0], 1)), X_test_scaled]



def sigmoid(z):
    """The Sigmoid function."""
    
    z = np.clip(z, -500, 500) 
    return 1 / (1 + np.exp(-z))

def gradient_descent_binary(X, y, lr, n_iterations):
    """Gradient Descent for a single Binary Logistic Regression classifier."""
    m, n = X.shape
    theta = np.zeros(n) 
    
    for _ in range(n_iterations):
        z = X @ theta
        h = sigmoid(z)
        
       
        gradient = (1 / m) * (X.T @ (h - y))
        
        
        theta = theta - lr * gradient
        
    return theta



lr = 0.1
n_iterations = 1000
n_features = X_train_b.shape[1]


theta_ovr = np.zeros((n_classes, n_features))


for k in range(n_classes): 
    
    y_binary = (y_train == k).astype(int)
    
    
    theta_ovr[k, :] = gradient_descent_binary(X_train_b, y_binary, lr, n_iterations)
    
    print(f"Classifier {k} (Class {iris.target_names[k]} vs. Rest) trained.")



def predict_ovr(X, theta_ovr):
    """Predicts class labels using the OvR strategy."""
    
    scores = X @ theta_ovr.T
    
    
    return np.argmax(scores, axis=1)


y_pred_test = predict_ovr(X_test_b, theta_ovr)

accuracy = accuracy_score(y_test, y_pred_test)


print(f"Multiclass Logistic Regression (OvR) Test Accuracy: {accuracy:.4f}")

Classifier 0 (Class setosa vs. Rest) trained.
Classifier 1 (Class versicolor vs. Rest) trained.
Classifier 2 (Class virginica vs. Rest) trained.
Multiclass Logistic Regression (OvR) Test Accuracy: 0.9111
